In [87]:
import clickhouse_connect
import pandas as pd

In [93]:
# Подключение к локальной БД clickhouse
if __name__ == '__main__':
    client = clickhouse_connect.get_client(
        host='rhnhiyh94p.europe-west4.gcp.clickhouse.cloud',
        user='default',
        password='        secure=True
    )
    print("Result:", client.query("SELECT 1").result_set[0][0])

Result: 1


Tasks:
1) To count the unique number of newcomers who visited us last week.
2) Select the top 3 popular authors of scores for the past month.
3) Calculate the number of sessions per user.

In [147]:
# Создание таблицы в базе данных
dropping_query = """ DROP TABLE IF EXISTS events; """
client.query(dropping_query)

create_events_table_query = """
    CREATE TABLE IF NOT EXISTS events (
        datetime DateTime,
        user_id UInt32,
        event String,
        score_id UInt32
    ) ENGINE = MergeTree()
    ORDER BY datetime;
    
    """
client.query(create_events_table_query)

insert_events_query = """
INSERT INTO events (datetime, user_id, event, score_id) VALUES
('2023-01-01 20:00:00', 1600664200, 'score_view', 419796001),
('2023-01-01 20:05:00', 1840964701, 'score_view', 419796001),
('2023-01-01 20:08:01', 1840964701, 'score_view', 219796002),
('2023-01-02 20:01:00', 1840964701, 'score_download', 219796002),
('2023-01-03 21:50:00', 2340964701, 'score_add_to_favorites', 219796002);
"""
client.query(insert_events_query)

events_query = """ SELECT * FROM events """
events = client.query(events_query)

# Преобразование результата в pandas DataFrame, просто чтобы привычно посмотреть в питоне
df_events = pd.DataFrame(events.result_set, columns=events.column_names)
df_events

,datetime,user_id,event,score_id
0,2023-01-01 20:00:00,1600664200,score_view,419796001
1,2023-01-01 20:05:00,1840964701,score_view,419796001
2,2023-01-01 20:08:01,1840964701,score_view,219796002
3,2023-01-02 20:01:00,1840964701,score_download,219796002
4,2023-01-03 21:50:00,2340964701,score_add_to_favorites,219796002


In [119]:
# Создание таблицы scores
create_scores_table_query = """
    CREATE TABLE IF NOT EXISTS scores (
        id UInt32,
        user_id UInt32
    ) ENGINE = MergeTree()
    ORDER BY id;
    
    """
client.query(create_scores_table_query)

# Вставка данных напрямую
insert_scores_query = """
    INSERT INTO scores (id, user_id) VALUES
    (419796001, 1600664200),
    (219796002, 1600664201);
    
    """
client.query(insert_scores_query)

scores_query = """ SELECT * FROM scores """
scores = client.query(scores_query)

# Преобразование результата в pandas DataFrame, просто чтобы привычно посмотреть в питоне
df_scores = pd.DataFrame(scores.result_set, columns=scores.column_names)
df_scores

,id,user_id
0,219796002,1600664201
1,419796001,1600664200


In [153]:
# Уникальные новички за последнюю неделю.
# To count the unique number of newcomers who visited us last week.

task_1 = '''
    SELECT 
        COUNT(DISTINCT user_id) AS unique_newcomers
    FROM 
        events
    WHERE TRUE
        AND datetime >= subtractDays(now(), 7)
        AND user_id NOT IN (
            SELECT DISTINCT user_id
            FROM events
            WHERE datetime < subtractDays(now(), 7)
        )
    '''
result_task_1 = client.query(task_1)

df_task_1 = pd.DataFrame(result_task_1.result_set, columns=result_task_1.column_names)

"""
Результат будет 0. 

subtractDays(now(), 7) возвращает актуальную даты минус 7 дней, 
потому что нам нужны были посетители "last week". 
Данные задачи содержат даты 2023го года, 
которые не подходят под datetime >= subtractDays(now(), 7).
То есть посетителей за последние 7 дней просто не было.
Я пробовал добавить данных со свежими датами, все работает.
Но решил их убрать, чтобы сохранить задачу как есть.

Вложенный запрос нужен, чтобы из основного запроса взять только новых посетителей, 
которых ранее не было в events.
"""

df_task_1

,unique_newcomers
0,0


In [204]:
# Топ-3 популярных автора
# Select the top 3 popular authors of scores for the past month.

task_2 = '''
    SELECT
        scores.user_id as author_id,
        COUNT(events.score_id) AS score_interactions
    FROM 
        events
    LEFT JOIN 
        scores 
    ON events.score_id = scores.id
    WHERE TRUE
        AND events.datetime >= subtractMonths(now(), 1)
    GROUP BY scores.user_id
    ORDER BY score_interactions DESC
    LIMIT 3
        
    '''
result_task_2 = client.query(task_2)
df_task_2 = pd.DataFrame(result_task_2.result_set, columns=result_task_2.column_names)

"""
Здесь также получается пустой результат из-за условия последнего месяца - subtractMonths(now(), 1).
Я еще не уверен, имеется ли в виду последний месяц как 30 прошедших дней последних, 
сделал по этому сценарию.

Если имеется в виду календарный один месяц, в плане если сейчас ноябрь, 
то выгружать нужно от начала до конца октябрь как прошлый месяц, 
то нужно было бы просто задать другой фильтр по дате.

"""

df_task_2

""


In [242]:
# Количество сессий на пользователя
# Calculate the number of sessions per user.

task_3 = '''
    SELECT
        user_id,
        COUNT(DISTINCT toStartOfInterval(datetime, INTERVAL 60 MINUTE)) AS sessions_count
    FROM events
    GROUP BY user_id
    
    '''
result_task_3 = client.query(task_3)
df_task_3 = pd.DataFrame(result_task_3.result_set, columns=result_task_3.column_names)

"""
Если условиться, что сессия длится 60 минут, а затем протухает, 
то запрос будет такой как выше.

Вопрос только к длительности сессии. 
В зависимости от этого нужно поменять 60 минут в запросе на нужное значение 
и запустить новый расчет.

Сейчас у пользователя 1840964701 две сессии. А событий в events было три.
Но два из них с разницей во времени несколько минут, 
поэтому, предположительно, были сделаны в рамках одной сессии.
А третье событие на другой день, вероятнее всего уже в новой сессии.
"""

df_task_3

,user_id,sessions_count
0,1840964701,2
1,2340964701,1
2,1600664200,1
